# Get 3D metrics for the DiffLinker generated poses

In [ ]:
import os
import glob
import sys
from typing import Union, Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem.MolStandardize import rdMolStandardize
import copy

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from utils.calc_sc_rdkit import calc_SC_RDKit_score
from utils.metrics_3d import lig_protein_clash_dist, lig_protein_clash_vdw, calc_torsion_energy, mcs_rmsd
from utils.chem_transforms import remove_atom_indices

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [ ]:
pdb = '7JTP'
method = 'difflinker'

## Load Data

### Generated data

In [ ]:
gen_folder = 'data/generated'
filepath = glob.glob(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid.csv'))
print(filepath)
df = pd.read_csv(filepath[0])
df.head()

In [ ]:
len(df)

#### Add generated pose to the data

In [ ]:
gen_pose_path = os.path.join(os.path.expanduser('~'), 'Documents/data/protacs/preprint_data/difflinker_results', pdb)

In [ ]:
def get_gen_confs(filepath: str, smi: str) -> pd.Series:
    mol_dirty = Chem.SDMolSupplier(os.path.join(gen_pose_path, filepath))[0]
    largest_Fragment = rdMolStandardize.LargestFragmentChooser()
    mol = largest_Fragment.choose(mol_dirty)
    mol_smi = Chem.MolFromSmiles(smi)
    assert mol_smi.HasSubstructMatch(mol), f'{smi} does not match {filepath}'
    return pd.Series({'gen_mol': mol}) 

In [ ]:
# store embedded conformers and vinardo scores in dataframe using apply
df[['gen_mol']] = df.progress_apply(lambda x: get_gen_confs(x['gen_filename'], x['protac_smiles']), axis=1)

### Xtal references

In [ ]:
xtal_folder = 'data/xtal_poses'
pdb_folder = os.path.join(xtal_folder, pdb, f'{pdb}_fragments')
xtal_protein_path = os.path.join(pdb_folder, f'{pdb}_protein.pdb')
# xtal_ext_linker = Chem.MolFromMolFile(os.path.join(pdb_folder,f'{pdb}_linker_extended.sdf'))
xtal_protac = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_protac.sdf'))
# xtal_linker = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_linker.sdf'))
xtal_anchor = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_anchor.sdf'))
xtal_warhead = Chem.MolFromMolFile(os.path.join(pdb_folder, f'{pdb}_warhead.sdf'))

## Get 3D metrics

In [ ]:
def gen_protac_conf_new(gen_filepath, protac_smiles, xtal_anchor, xtal_warhead):
    filepath = os.path.join(gen_pose_path, gen_filepath)
    mol_protac = Chem.MolFromSmiles(protac_smiles)
    mol_gen = Chem.SDMolSupplier(filepath)[0]
    largest_Fragment = rdMolStandardize.LargestFragmentChooser()
    mol_gen = largest_Fragment.choose(mol_gen)
    mol_gen_smi = Chem.MolToSmiles(mol_gen, isomericSmiles=False)
    protac_smiles_flat = Chem.MolToSmiles(mol_protac, isomericSmiles=False)
    if mol_gen_smi != protac_smiles_flat:
        xtal_anc_copy = copy.deepcopy(xtal_anchor)
        xtal_wrh_copy = copy.deepcopy(xtal_warhead)
        mol_gen_coords = []
        for i, atom in enumerate(mol_gen.GetAtoms()):
            atom.SetAtomMapNum(i+1)
            mol_gen_coords.append(mol_gen.GetConformer().GetAtomPosition(i))
        dict_gen = {a.GetAtomMapNum(): a.GetIdx() for a in mol_gen.GetAtoms()}
        xtal_anc_cp_coords = []
        for i, atom in enumerate(xtal_anc_copy.GetAtoms()):
            atom.SetAtomMapNum(i+100)
            xtal_anc_cp_coords.append(xtal_anc_copy.GetConformer().GetAtomPosition(i))
        dict_anc = {a.GetAtomMapNum(): a.GetIdx() for a in xtal_anc_copy.GetAtoms()}
        xtal_wrh_cp_coords = []
        for i, atom in enumerate(xtal_wrh_copy.GetAtoms()):
            atom.SetAtomMapNum(i+200)
            xtal_wrh_cp_coords.append(xtal_wrh_copy.GetConformer().GetAtomPosition(i))
        dict_wrh = {a.GetAtomMapNum(): a.GetIdx() for a in xtal_wrh_copy.GetAtoms()}
        # check if matching coordinates between xtal and gen
        overlap_mapNum_anc = []
        dict_gen2anc = {}
        for i, coord in enumerate(mol_gen_coords):
            for j, anc_coord in enumerate(xtal_anc_cp_coords):
                if coord.x == anc_coord.x and coord.y == anc_coord.y and coord.z == anc_coord.z:
                    overlap_mapNum_anc.append(mol_gen.GetAtomWithIdx(i).GetAtomMapNum())
                    dict_gen2anc[mol_gen.GetAtomWithIdx(i).GetAtomMapNum()] = xtal_anc_copy.GetAtomWithIdx(j).GetAtomMapNum()
        overlap_mapNum_wrh = []
        dict_gen2wrh = {}
        for i, coord in enumerate(mol_gen_coords):
            for j, wrh_coord in enumerate(xtal_wrh_cp_coords):
                if coord.x == wrh_coord.x and coord.y == wrh_coord.y and coord.z == wrh_coord.z:
                    overlap_mapNum_wrh.append(mol_gen.GetAtomWithIdx(i).GetAtomMapNum())
                    dict_gen2wrh[mol_gen.GetAtomWithIdx(i).GetAtomMapNum()] = xtal_wrh_copy.GetAtomWithIdx(j).GetAtomMapNum()
        # find the bond between overlap and not overlap
        map_in_anc = None
        map_in_wrh = None
        for bond in mol_gen.GetBonds():
            if bond.GetBeginAtom().GetAtomMapNum() in overlap_mapNum_anc and bond.GetEndAtom().GetAtomMapNum() not in overlap_mapNum_anc:
                map_in_anc = bond.GetBeginAtom().GetAtomMapNum()
                map_not_in_anc = bond.GetEndAtom().GetAtomMapNum()
                break
            if bond.GetEndAtom().GetAtomMapNum() in overlap_mapNum_anc and bond.GetBeginAtom().GetAtomMapNum() not in overlap_mapNum_anc:
                map_in_anc = bond.GetEndAtom().GetAtomMapNum()
                map_not_in_anc = bond.GetBeginAtom().GetAtomMapNum()
                break
        for bond in mol_gen.GetBonds():
            if bond.GetBeginAtom().GetAtomMapNum() in overlap_mapNum_wrh and bond.GetEndAtom().GetAtomMapNum() not in overlap_mapNum_wrh:
                map_in_wrh = bond.GetBeginAtom().GetAtomMapNum()
                map_not_in_wrh = bond.GetEndAtom().GetAtomMapNum()
                break
            if bond.GetEndAtom().GetAtomMapNum() in overlap_mapNum_wrh and bond.GetBeginAtom().GetAtomMapNum() not in overlap_mapNum_wrh:
                map_in_wrh = bond.GetEndAtom().GetAtomMapNum()
                map_not_in_wrh = bond.GetBeginAtom().GetAtomMapNum()
                break
        if map_in_anc is None or map_in_wrh is None:
            return pd.Series({'gen_mol': None, 'ori_gen_ptc_filename': None})  
        # remove atoms of overlap in gen
        wrh_ind_toremove = [dict_gen[x] for x in overlap_mapNum_wrh]
        # sort in reverse
        wrh_ind_toremove.sort(reverse=True)
        mol_gen_adapt = remove_atom_indices(mol_gen, wrh_ind_toremove)
        dict_gen = {a.GetAtomMapNum(): a.GetIdx() for a in mol_gen_adapt.GetAtoms()}
        anc_ind_toremove = [dict_gen[x] for x in overlap_mapNum_anc]
        # sort in reverse
        anc_ind_toremove.sort(reverse=True)
        mol_gen_adapt = remove_atom_indices(mol_gen_adapt, anc_ind_toremove)
        # remove atom if hydrogen at attechment point in xtal
        index_attach = dict_anc[dict_gen2anc[map_in_anc]]
        # get neighbors for index_attach
        xtal_anc_copy = Chem.AddHs(xtal_anc_copy)
        nbrs = xtal_anc_copy.GetAtomWithIdx(index_attach).GetNeighbors()
        for nbr in nbrs:
            if nbr.GetSymbol() == 'H':
                xtal_anc_copy = remove_atom_indices(xtal_anc_copy, [nbr.GetIdx()])
                break
        index_attach = dict_wrh[dict_gen2wrh[map_in_wrh]]
        # get neighbors for index_attach
        xtal_wrh_copy = Chem.AddHs(xtal_wrh_copy)
        nbrs = xtal_wrh_copy.GetAtomWithIdx(index_attach).GetNeighbors()
        for nbr in nbrs:
            if nbr.GetSymbol() == 'H':
                xtal_wrh_copy = remove_atom_indices(xtal_wrh_copy, [nbr.GetIdx()])
                break
        combo_gen_anc = Chem.CombineMols(mol_gen_adapt, xtal_anc_copy)
        dict_combo = {a.GetAtomMapNum(): a.GetIdx() for a in combo_gen_anc.GetAtoms()}
        combo_gen_anc_edit = Chem.EditableMol(combo_gen_anc)
        combo_gen_anc_edit.AddBond(dict_combo[map_not_in_anc], dict_combo[dict_gen2anc[map_in_anc]], Chem.BondType.SINGLE)
        combo_gen_anc = combo_gen_anc_edit.GetMol()
        combo_all = Chem.CombineMols(combo_gen_anc, xtal_wrh_copy)
        dict_combo = {a.GetAtomMapNum(): a.GetIdx() for a in combo_all.GetAtoms()}
        combo_all_edit = Chem.EditableMol(combo_all)
        combo_all_edit.AddBond(dict_combo[map_not_in_wrh], dict_combo[dict_gen2wrh[map_in_wrh]], Chem.BondType.SINGLE)
        combo_all = combo_all_edit.GetMol()
        combo_all =Chem.RemoveHs(combo_all)
        for a in combo_all.GetAtoms():
            a.SetAtomMapNum(0)
        flat_combo = Chem.MolToSmiles(combo_all, isomericSmiles=False)
        if flat_combo != protac_smiles_flat:
            print('ERROR: flat_combo != protac_smiles_flat')
            return pd.Series({'gen_mol': None, 'ori_gen_ptc_filename': None})  
        gen_filename = gen_filepath.split('.')[0]
        new_filepath = os.path.join(gen_pose_path, f'{gen_filename}_expanded.sdf')    
        Chem.MolToMolFile(combo_all, new_filepath)
        return pd.Series({'gen_mol': combo_all, 'ori_gen_ptc_filename': new_filepath})


In [ ]:
df[['gen_mol', 'ori_gen_ptc_filename']] = df.progress_apply(lambda x: gen_protac_conf_new(x['gen_filename'], x['protac_smiles'], xtal_anchor, xtal_warhead), axis=1)

In [ ]:
df_fil = df[~df.ori_gen_ptc_filename.isnull()]
len(df_fil)

In [ ]:
# percent failed
(len(df)-len(df_fil))/len(df)*100

### SC RDKit

In [ ]:
df_fil['ori_sc_rdkit'] = df_fil.apply(lambda x: calc_SC_RDKit_score(x.gen_mol, xtal_protac), axis=1)
print(df_fil['ori_sc_rdkit'].describe())

In [ ]:
print(f'fraction of conformers with SC_RDKIT > 0.7: {(len(df_fil[df_fil["ori_sc_rdkit"] > 0.7])/len(df_fil)*100):.4f}')
print(f'fraction of conformers with SC_RDKIT > 0.8: {(len(df_fil[df_fil["ori_sc_rdkit"] > 0.8])/len(df_fil)*100):.4f}')
print(f'fraction of conformers with SC_RDKIT > 0.9: {(len(df_fil[df_fil["ori_sc_rdkit"] > 0.9])/len(df_fil)*100):.4f}')
print(f'average SC_RDKIT: {df_fil["ori_sc_rdkit"].mean(skipna=True):.4f}')

### Clashes with protein

In [ ]:
df_fil['ori_clashes_cutoff'] = df_fil.apply(lambda x: lig_protein_clash_dist(xtal_protein_path, os.path.join(gen_pose_path, x.ori_gen_ptc_filename.split('/')[-1])), axis=1)
df_fil['ori_clashes_vdw'] = df_fil.apply(lambda x: lig_protein_clash_vdw(xtal_protein_path, os.path.join(gen_pose_path, x.ori_gen_ptc_filename.split('/')[-1])), axis=1)
print(f'average clashes_cutoff: {df_fil["ori_clashes_cutoff"].mean(skipna=True):.4f}')
print(f'average clashes_vdw: {df_fil["ori_clashes_vdw"].mean(skipna=True):.4f}')

### Torsion energy

In [ ]:
df_fil['ori_E_torsion'] = df_fil.apply(lambda x: calc_torsion_energy(os.path.join(gen_pose_path, x.ori_gen_ptc_filename.split('/')[-1])), axis=1)
print(f'average E_torsion: {df_fil["ori_E_torsion"].mean(skipna=True):.4f}')

In [ ]:
len(df_fil)

In [ ]:
df_fil.columns, df.columns

In [ ]:
df_loss = df[df.ori_gen_ptc_filename.isnull()]
len(df_loss)

In [ ]:
# concat
df_save = pd.concat([df_fil, df_loss], axis=0)
df_save.reset_index(drop=True, inplace=True)
df_save.head()

In [ ]:
# drop gen_mol column
df_save = df_save.drop(columns=['gen_mol'])
df_save.to_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid.csv'), index=False)

#### reinsert corrected value to filtered df

In [ ]:
len(df_save)

In [ ]:
df_smaller = pd.read_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid_fil.csv'))
df_smaller.head()

In [ ]:
df_smaller.columns

In [ ]:
columns_replace = ['ori_E_torsion', 'ori_clashes_cutoff', 'ori_clashes_vdw',
       'ori_gen_ptc_filename', 'ori_sc_rdkit']
for col in columns_replace:
    # store df_save values in df_smaller by  mapping ID
       df_smaller[col] = df_smaller['ID'].map(df_save.set_index('ID')[col])

In [ ]:
### correct! add to both valid and valid_fil
df_smaller.to_csv(os.path.join(gen_folder, f'{pdb}_sampled_{method}_valid_fil.csv'), index=False)